# Part 2 - Views and Visualization

## Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sqlite3

## Function to retrieve Table

In [2]:
def retrieve_table(table_name: str, db: str):
    conn = sqlite3.connect(f"../{db}.db")
    c = conn.cursor()
    data = c.execute(f"""SELECT * FROM {table_name}""").fetchall()
    columns = c.execute(f"""PRAGMA table_info({table_name})""").fetchall()
    columns = [columns[i][1] for i in range(0, len(columns))]
    return pd.DataFrame(data, columns=columns)

In [3]:
sql_tables = {}
for table_name in ['artist','album','track','track_feature']:
    sql_tables[table_name] = retrieve_table(table_name, "spotify")

## Create Views

### Common Functions

In [4]:
def add_view(query: str, view_name: str, db: str):
    conn = sqlite3.connect(f"../{db}.db")
    c = conn.cursor()
    c.execute(f"DROP VIEW IF EXISTS {view_name}")
    c.execute(f"""CREATE VIEW {view_name} AS {query}""")

In [5]:
def retrieve_query_pd(query: str, columns: list, db: str):
    conn = sqlite3.connect(f"../{db}.db")
    c = conn.cursor()
    data = c.execute(query)
    data = data.fetchall()
    return pd.DataFrame(data, columns=columns)

In [6]:
def plot_spotify_bar(df: str, y: list, width: list, title: str, xlabel: str, filename_to_save: str):
    
    fig, ax = plt.subplots(figsize = (10,9))
        
    rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))
    color=plt.get_cmap("viridis")(rescale(width))
    
    plt.barh(y = y, width = width, color = color)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.grid(axis = 'x')
    plt.savefig(f'../views/{filename_to_save}.png', bbox_inches='tight')

### Top songs by artist in terms of duration_ms

In [7]:
query_top_10_songs_by_artist_by_duration = f"""
    WITH TOPTEN AS (SELECT artist.artist_name,
                          track.song_name,
                          track.duration_ms,
                          ROW_NUMBER() over (
                                PARTITION BY artist.artist_name
                                order by track.duration_ms DESC
                            ) AS RowNo 
                    FROM artist JOIN album ON artist.artist_id = album.artist_id
                        JOIN track ON album.album_id = track.album_id
                    )
    SELECT artist_name, song_name, duration_ms
    FROM TOPTEN WHERE RowNo <= 10;
    """


In [8]:
add_view(query_top_10_songs_by_artist_by_duration, 'top_10_songs_by_artist_by_duration', 'spotify')

In [9]:
retrieve_query_pd("SELECT * FROM top_10_songs_by_artist_by_duration",['artist_name','song_name','duration_ms'], 'spotify').song_name.sort_values().values



array(["(I Can't Get No) Satisfaction",
       '(I Got That) Boom Boom (feat. Ying Yang Twins)',
       '5446 Thats My Number/ Ball And Chain', '5:24',
       'A Rush of Blood to the Head', 'Addicted', 'Adicto', 'Afire Love',
       'Alaska', 'Alguien Como Tú', 'All Too Well',
       "All Too Well (10 Minute Version) (Taylor's Version) (From The Vault)",
       "All Too Well (Taylor's Version)", 'Ambulancia', 'Andrea',
       'Arabesque', 'Army of One', 'Bad Romance', 'Batches & Cookies',
       'Beautiful Goodbye',
       "Better Man (Taylor's Version) (From The Vault)",
       'Big Fat Bass (feat. will.i.am)', 'Bloodlines', 'Bloodstream',
       'Broken Hearted', 'Bus Passes and Happy Meals', 'Callaita',
       'Cherry', 'Cinema', 'Closure', 'Coloratura',
       'Colours - 2016 Remaster', 'Colours - Live 1990', 'Contigo',
       'Dagger One Is Hit / Time To Let Go', 'Death and All His Friends',
       'Desnudo', "Doin' Time - Eerie Splendor Remix",
       "Doin' Time - Eerie Splendo

### Top artists in the database by # of followers

In [10]:
query_top_20_artists_by_followers = """SELECT artist_name, followers
                                        FROM artist
                                        ORDER BY 2 DESC"""

In [11]:
add_view(query_top_20_artists_by_followers, 'top_20_artists_by_followers', 'spotify')

In [12]:
retrieve_query_pd("SELECT * FROM top_20_artists_by_followers",['artist_name','followers'], 'spotify')

,artist_name,followers
0,Ed Sheeran,102584457
1,Billie Eilish,68915938
2,Taylor Swift,58988141
3,Bad Bunny,56219718
4,Queen,42790568
5,Adele,42433872
6,Coldplay,38595052
7,Maroon 5,37922762
8,Beyoncé,32173211
9,Shakira,25002007


### Top songs by artist in terms of tempo

In [13]:
query_top_10_songs_by_artist_by_tempo = """
    WITH TOPTEN AS (SELECT artist.artist_name,
                          track.song_name,
                          track_feature.tempo,
                          ROW_NUMBER() over (
                                PARTITION BY artist.artist_name
                                order by track_feature.tempo DESC
                            ) AS RowNo 
                    FROM artist JOIN album ON artist.artist_id = album.artist_id
                        JOIN track ON album.album_id = track.album_id
                        JOIN track_feature ON track.track_id = track_feature.track_id
                    )
    SELECT artist_name, song_name, tempo
    FROM TOPTEN WHERE RowNo <= 10;
    """


In [14]:
add_view(query_top_10_songs_by_artist_by_tempo, 'top_10_songs_by_artist_by_tempo', 'spotify')

In [27]:
retrieve_query_pd("SELECT * FROM top_10_songs_by_artist_by_tempo where artist_name = 'Taylor Swift'",['artist_name','song_name','tempo'], 'spotify')

,artist_name,song_name,tempo
0,Taylor Swift,State Of Grace (Acoustic Version) (Taylor's Ve...,208.918
1,Taylor Swift,Soon You’ll Get Better (feat. The Chicks),207.476
2,Taylor Swift,Untouchable (Taylor’s Version),200.391
3,Taylor Swift,All Too Well (Taylor's Version),185.972
4,Taylor Swift,White Horse (Taylor’s Version),185.262
5,Taylor Swift,Wonderland,184.014
6,Taylor Swift,ME! (feat. Brendon Urie of Panic! At The Disco),182.162
7,Taylor Swift,Our Song,178.013
8,Taylor Swift,We Are Never Ever Getting Back Together (Taylo...,172.014
9,Taylor Swift,Superstar (Taylor’s Version),171.862


### Top album for each artist by average song energy

In [110]:
query_top_1_album_by_artist_by_mean_song_energy = """
    WITH TOP AS (SELECT artist.artist_name,
                          album.album_name,
                          track_feature.energy as mean_energy,
                          ROW_NUMBER() over (
                                PARTITION BY album.album_name
                                order by track_feature.energy DESC 
                            ) AS RowNo 
                    FROM artist JOIN album ON artist.artist_id = album.artist_id
                        JOIN track ON album.album_id = track.album_id
                        JOIN track_feature ON track.track_id = track_feature.track_id
                    
                    )
    SELECT artist_name, album_name, mean_energy
    FROM TOP WHERE RowNo <= 1;
    """


In [96]:
query_top_1_album_by_artist_by_mean_song_energy = """
    SELECT artist.artist_name,
                          album.album_name,
                          AVG(track_feature.energy) as mean_energy
                    FROM artist JOIN album ON artist.artist_id = album.artist_id
                        JOIN track ON album.album_id = track.album_id
                        JOIN track_feature ON track.track_id = track_feature.track_id
                    GROUP BY 1, 2
                    ORDER BY 3 DESC
    """


In [131]:
query_top_1_album_by_artist_by_mean_song_energy = """
    WITH mean_energy as (SELECT artist.artist_name,
                          album.album_name,
                          AVG(track_feature.energy) as mean_energy
                    FROM artist JOIN album ON artist.artist_id = album.artist_id
                        JOIN track ON album.album_id = track.album_id
                        JOIN track_feature ON track.track_id = track_feature.track_id
                    GROUP BY 1, 2
                    ORDER BY 3 DESC),
    mean_energy_row_no AS (SELECT *, ROW_NUMBER() over (PARTITION BY artist_name
                                                        order by mean_energy DESC
                                                        ) AS RowNo
                            FROM mean_energy)
    SELECT *
    FROM mean_energy_row_no 
    """


In [121]:
retrieve_query_pd(query_top_1_album_by_artist_by_mean_song_energy, ['artist_name', 'album_name', 'mean_energy'], 'spotify')

ValueError: 3 columns passed, passed data had 4 columns

In [132]:
retrieve_query_pd(query_top_1_album_by_artist_by_mean_song_energy, ['artist_name', 'album_name', 'mean_energy','SDFS'], 'spotify')

OperationalError: a GROUP BY clause is required before HAVING

In [86]:
query_top_1_album_by_artist_by_mean_song_energy = """
    SELECT artist_name, album.album_name, AVG(track_feature.energy) as mean_energy,
    FROM artist JOIN album ON artist.artist_id = album.artist_id
                        JOIN track ON album.album_id = track.album_id
                        JOIN track_feature ON track.track_id = track_feature.track_id
    GROUP BY 1, 2
    """


In [67]:
add_view(query_top_10_songs_by_artist_by_tempo, 'top_10_songs_by_artist_by_tempo', 'spotify')

In [39]:
retrieve_query_pd("SELECT * FROM top_10_songs_by_artist_by_tempo where artist_name = 'Taylor Swift'",['artist_name','song_name','tempo'], 'spotify')

,artist_name,song_name,tempo
0,Taylor Swift,State Of Grace (Acoustic Version) (Taylor's Ve...,208.918
1,Taylor Swift,Soon You’ll Get Better (feat. The Chicks),207.476
2,Taylor Swift,Untouchable (Taylor’s Version),200.391
3,Taylor Swift,All Too Well (Taylor's Version),185.972
4,Taylor Swift,White Horse (Taylor’s Version),185.262
5,Taylor Swift,Wonderland,184.014
6,Taylor Swift,ME! (feat. Brendon Urie of Panic! At The Disco),182.162
7,Taylor Swift,Our Song,178.013
8,Taylor Swift,We Are Never Ever Getting Back Together (Taylo...,172.014
9,Taylor Swift,Superstar (Taylor’s Version),171.862


## Data Visualizations

## Merge tables into songs dataframe

In [40]:
songs = pd.merge(sql_tables['track_feature'], sql_tables['track'], on="track_id", how="outer", suffixes=('_feature','_track'))

songs = pd.merge(songs, sql_tables['album'], on="album_id", how="outer",suffixes=('_track','_album'))

songs = pd.merge(songs, sql_tables['artist'], on="artist_id", how="outer",suffixes=('_album','_artist'))

In [41]:
songs

,track_id,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,type_feature,valence,...,album_uri,artist_id,artist_name,external_url,genre,image_url_artist,followers,popularity,type_artist,artist_uri
0,4yugZvBYaoREkJKtbG08Qr,0.575,0.670,0.000005,0.1290,-10.390,0.0318,139.191,audio_features,0.740,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8613819,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL
1,436yrzQWA32vb1sTZKXg9r,0.555,0.568,0.001670,0.1160,-12.205,0.0341,98.507,audio_features,0.583,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8613819,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL
2,1uNrJfFUItP86a1m19Yr5n,0.537,0.770,0.000046,0.0970,-8.896,0.0534,138.471,audio_features,0.644,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8613819,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL
3,2j7KL0fcBKEh6P8z0VxIVr,0.559,0.251,0.005730,0.0866,-14.288,0.0262,109.551,audio_features,0.285,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8613819,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL
4,59OUN6rPXLBK6bkhhwQkP5,0.647,0.625,0.006360,0.2760,-9.347,0.0288,135.046,audio_features,0.679,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8613819,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,2OlxJKrbyLmzAO14EgV53K,0.891,0.506,0.000016,0.0692,-1.980,0.1590,148.016,audio_features,0.938,...,spotify:album:1UTDgnpHmthIsdzSxbhpV2,28gNT5KBp7IjEOQoevXf9N,Camilo,https://open.spotify.com/artist/28gNT5KBp7IjEO...,colombian pop,https://i.scdn.co/image/ab6761610000e5ebc85ae7...,16418116,81,artist,spotify:artist:28gNT5KBp7IjEOQoevXf9N
2257,7mCC2SBkb5uwmQ4Ozewtbc,0.751,0.638,0.000000,0.3570,-5.569,0.1600,149.943,audio_features,0.791,...,spotify:album:1UTDgnpHmthIsdzSxbhpV2,28gNT5KBp7IjEOQoevXf9N,Camilo,https://open.spotify.com/artist/28gNT5KBp7IjEO...,colombian pop,https://i.scdn.co/image/ab6761610000e5ebc85ae7...,16418116,81,artist,spotify:artist:28gNT5KBp7IjEOQoevXf9N
2258,0j3ULEzS14shcTGdiYwsOI,0.812,0.502,0.000000,0.1530,-4.763,0.0632,90.057,audio_features,0.883,...,spotify:album:1UTDgnpHmthIsdzSxbhpV2,28gNT5KBp7IjEOQoevXf9N,Camilo,https://open.spotify.com/artist/28gNT5KBp7IjEO...,colombian pop,https://i.scdn.co/image/ab6761610000e5ebc85ae7...,16418116,81,artist,spotify:artist:28gNT5KBp7IjEOQoevXf9N
2259,3CzgTXrAROSCB6bd1bm5mJ,0.805,0.484,0.000000,0.0979,-4.984,0.1490,143.873,audio_features,0.653,...,spotify:album:1UTDgnpHmthIsdzSxbhpV2,28gNT5KBp7IjEOQoevXf9N,Camilo,https://open.spotify.com/artist/28gNT5KBp7IjEO...,colombian pop,https://i.scdn.co/image/ab6761610000e5ebc85ae7...,16418116,81,artist,spotify:artist:28gNT5KBp7IjEOQoevXf9N


In [42]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2261 entries, 0 to 2260
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   track_id            2261 non-null   object 
 1   danceability        2261 non-null   float64
 2   energy              2261 non-null   float64
 3   instrumentalness    2261 non-null   float64
 4   liveness            2261 non-null   float64
 5   loudness            2261 non-null   float64
 6   speechiness         2261 non-null   float64
 7   tempo               2261 non-null   float64
 8   type_feature        2261 non-null   object 
 9   valence             2261 non-null   float64
 10  song_uri_feature    2261 non-null   object 
 11  song_name           2261 non-null   object 
 12  external_url_track  2261 non-null   object 
 13  duration_ms         2261 non-null   int64  
 14  explicit            2261 non-null   int64  
 15  disc_number         2261 non-null   int64  
 16  type_t

In [43]:
songs.groupby("artist_name").nunique().iloc[:, 0:10]

,track_id,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,type_feature,valence
artist_name,,,,,,,,,,
Adele,46,45,46,20,40,46,45,46,1,46
Bad Bunny,92,84,80,60,78,92,87,92,1,83
Beyoncé,205,177,181,136,184,203,183,202,1,177
Billie Eilish,30,30,29,28,29,30,29,30,1,30
Britney Spears,125,104,111,72,112,124,115,124,1,115
Camilo,32,32,32,5,32,32,31,32,1,32
Coldplay,106,92,100,103,92,106,83,106,1,101
Eagles,104,95,97,96,93,103,86,104,1,96
Ed Sheeran,85,78,83,31,78,85,80,84,1,80
